In [9]:
import pandas as pd
import time
from datetime import datetime
from pandas import ExcelWriter 
pd.__version__




'1.0.3'

In [10]:
# temp=1588744463872/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

NameError: name 'temp' is not defined

In [3]:
temp=1588744479826/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

'2020-05-06 13:54:39'

In [4]:
    # Total number of seconds between dates

In [135]:
class JTLReader:
    def __init__(self,jtlPath):
        self.jtlPath=jtlPath
        self.jtlColumns = ['timeStamp', 'elapsed','label','dataType','bytes','URL','allThreads','success','responseCode','responseMessage']
        %time self.df_Jtl=pd.read_csv(self.jtlPath,usecols=self.jtlColumns,dtype={'label':'category'})#dtype={'label':'category'},nrows=195222    
        self.shape=self.df_Jtl.shape
        self.rows=self.shape[0]
        self.rows
        """Conver Epoc time to Actual Time Start"""
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')
        """Conver Epoc time to Actual Time End"""
        self.filt=(pd.isnull(self.df_Jtl['URL'])) #& (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[self.filt]
        self.df_jtl_HTTPReqeust=self.df_Jtl.loc[~self.filt]
        
        self.error_filt=(self.df_jtl_HTTPReqeust['success']==False)
        self.df_HttpErrors=self.df_jtl_HTTPReqeust.loc[self.error_filt]
        
        
        
      
        
    """def getTransactionControllerData(self):
        print("Hello shiva")
        filt=(pd.isnull(self.df_Jtl['URL'])) & (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[filt]
        return self.df_jtl_TC"""
        
       
        
    def converTimeStamp(self):
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]') 
        
    def ms_to_sec(self):
         self.df_Jtl['elapsed']=((self.df_Jtl['elapsed'])/1000)
        
        
    def overAllResponseTimeSummary(self):
        self.df_overAllResponseTimeSummary=self.df_jtl_TC[['timeStamp','elapsed']]
        return self.df_overAllResponseTimeSummary
       
    def listOfErrors(self):
        return self.df_Jtl['responseCode'].value_counts()
    
    def shapeOfDaraFrame(self):
        return self.df_Jtl.shape
    
    def columnType(self,columnName):
        self.columnName=columnName
        return self.df_Jtl[self.columnName].dtype
    
    def getPeriod(self):
        self.startTime=self.df_Jtl['timeStamp'][0]
        self.endRow=self.df_Jtl['timeStamp'].shape[0]
        self.endTime=self.df_Jtl['timeStamp'][self.endRow-1]
        return f'{self.startTime} : {self.endTime}'
    
    def getDuration(self):
        self.startTime= self.df_Jtl['timeStamp'][0]
        self.endRow= self.df_Jtl['timeStamp'].shape[0]
        self.endTime= self.df_Jtl['timeStamp'][self.endRow-1]#-1
        self.executionStartTime=datetime(self.startTime.year,self.startTime.month,self.startTime.day,self.startTime.hour,self.startTime.minute,self.startTime.second)
        self.executionEndTime=datetime(self.endTime.year,self.endTime.month,self.endTime.day,self.endTime.hour,self.endTime.minute,self.endTime.second)
        self.timeDiff = self.executionEndTime-self.executionStartTime
        self.duration_in_s = self.timeDiff.total_seconds()
        self.duration=time.strftime("%H:%M:%S", time.gmtime(self.duration_in_s))
        return self.duration
    
    def getMaxRunningVusers(self):
        self.allThread=self.df_Jtl['allThreads'].max()
        return self.allThread 
    
      
    def getTotalHits(self):
        return self.df_jtl_HTTPReqeust.shape[0]
    
    def getAvgHitsPerSecond(self):
        return self.df_jtl_HTTPReqeust.shape[0]/self.duration_in_s
    
    def getTotalThroughput(self):
        self.totalThroughput=self.df_jtl_TC['bytes'].sum()
        return self.totalThroughput
    
    def getAvgThroughputPerSecond(self):
        self.avgThroughput=(self.df_jtl_TC['bytes'].sum())/self.duration_in_s
        return self.avgThroughput
    
    def getTotalPassFailFreqList(self):
        self.passFail_freq=self.df_jtl_TC['success'].value_counts().to_frame()
        #self.passFailFrequencyeqList=self.passFail_freq.index.tolist()
        
        return self.passFail_freq

        
    
    

    def getAnalysisSummary(self):
        self.period=self.getPeriod()
        self.duration=self.getDuration()
        self.maxRunningVusers=self.getMaxRunningVusers()
        self.totalThroughput=self.getTotalThroughput()
        self.avgThroughputPerSecond=self.getAvgThroughputPerSecond()
        self.totalHits=self.getTotalHits()
        self.avgHitsPerSecond=self.getAvgHitsPerSecond()
        
        self.df_AnalysisSummary=pd.DataFrame(
            {
                'Data':[
                        'Period','Duration','Maximum Running Vusers','Total bytes',
                        'Average bytes/second','TotalHits','Average Hits Per Second',
                        'Total Error','Total Pass','Total Failed'
                ],
                'Values':[
                            self.period,self.duration,self.maxRunningVusers,self.totalThroughput,
                            self.avgThroughputPerSecond,self.totalHits,self.avgHitsPerSecond,
                            'NA',self.getTotalPassFailFreqList(),'NA'
                ]
                
            }
        )
        return self.df_AnalysisSummary
    
    def responseTimeSummary(self):
        self.df_responseTimeSummary=self.df_jtl_TC[['label','elapsed']]
        self.df_txnGroup=self.df_responseTimeSummary.groupby(['label'],observed=True )  #observed=True      
        self.df_txnGroup=self.df_txnGroup['elapsed'].describe(percentiles=[.90])#self.df_txnGroup.describe(percentiles=[.90])
        self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(0)#self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(1)
        self.df_txnGroup = self.df_txnGroup[['count','min','mean','90%','max',]]
        self.df_txnGroup.rename(columns={'count':'#Samples','min':'Min','mean':'Average','90%':'90 Percentile','max':'Max'},inplace=True)
        ##self.df_txnGroup=self.df_txnErrorGroup.reset_index() """To be enable if you run describe () on data frame. Now we are running on specific column"""
        ##self.df_txnGroup['throughput']=self.df_txnGroup['count']/self.duration_in_s
        ##self.validTxn_filt=(self.df_txnGroup['count']>0)
        self.df_responseTimeSummaryOutput=self.df_txnGroup #self.df_txnGroup[self.validTxn_filt]
       
        return self.df_responseTimeSummaryOutput
    
    def getPassFailSummary(self):
        self.df_passFailSummary=self.df_jtl_TC[['label','success']]
        self.df_passFailGroup=self.df_passFailSummary.groupby(['label'],observed=True)#whe we use observed=True-->refer https://stackoverflow.com/questions/54922327/valueerror-operands-could-not-be-broadcast-together-with-shape-when-calling-pan?answertab=votes#tab-top
        self.df_txnGroup=self.df_passFailGroup['success'].value_counts()
        self.df_txnGroupSlice=self.df_txnGroup.loc[:,]
        self.df_passFailSummaryOutput=self.df_txnGroupSlice.unstack()
        self.df_passFailSummaryOutput=self.df_passFailSummaryOutput.fillna(0)
        self.df_passFailSummaryOutput.rename(columns={True:'Pass',False:'Fail'},inplace=True)
        return self.df_passFailSummaryOutput
        
    def getFinalResponseSummary(self):
        rs=pd.merge(self.df_responseTimeSummaryOutput,self.df_passFailSummaryOutput,left_index=True,right_index=True)
        rs['Error%']=((rs['Fail'])/(rs['#Samples']))*100 
        rs=rs[['#Samples','Pass','Fail','Error%','Min','Average','90 Percentile','Max']]#'Fail','Error%'
        return rs    
    
    def getRunnerVuserGraphData(self):
        '''T=min,S=Seconds'''       
        self.df_vusers=self.df_jtl_TC[['timeStamp','allThreads']]
        self.df_vusers.sort_values(by=['timeStamp'])
        self.df_vusers.set_index('timeStamp',inplace=True)
        self.df_vusers_summary = pd.DataFrame()
        self.df_vusers_summary['Vusers']=self.df_vusers['allThreads'].resample('30T').max()     
        return self.df_vusers_summary
         
        
    
    def getHitsPersecondGraphData(self):
        '''T=min,S=Seconds'''
        #self.df_jtl_HTTPReqeust['hits']=1
        self.df_hits=self.df_jtl_HTTPReqeust[['timeStamp','URL']]#self.df_jtl_HTTPReqeust[['timeStamp','hits']]
        self.df_hits.sort_values(by=['timeStamp'])
        self.df_hits.set_index('timeStamp',inplace=True)
        self.df_hits_summary = pd.DataFrame()
        self.df_hits_summary['hitsCount']=self.df_hits['URL'].resample('30T').count()     
        return self.df_hits_summary
    
    def getThrouhputPersecondGraphData(self):
        '''T=min,S=Seconds'''
        self.df_throughput=self.df_jtl_TC[['timeStamp','bytes']]
        self.df_throughput.sort_values(by=['timeStamp'])
        self.df_throughput.set_index('timeStamp',inplace=True)
        self.df_throughput_summary = pd.DataFrame()
        self.df_throughput_summary['Throughput']=self.df_throughput['bytes'].resample('30T').sum()     
        return self.df_throughput_summary
    
    def getOverallResponseTime(self):
        return self.df_jtl_TC[['timeStamp','elapsed','label']]
    
    def temp(self):
        self.slicer_grp=self.df_jtl_TC[['timeStamp','elapsed','label']]
        self.slicer_grp.sort_values(by=['timeStamp'])
        self.slicer_grp.set_index('timeStamp',inplace=True)
        self.df_response_summary_slicer = pd.DataFrame()
        self.df_slicer_txnGroup=self.slicer_grp.groupby(['label'],observed=True )
        self.df_response_summary_slicer=self.df_slicer_txnGroup['elapsed'].resample('30T').agg(['min','mean','max','count'])
        return self.df_response_summary_slicer
    
    def getResponseMessage(self):
         return self.df_HttpErrors
        
        
    
    
      
obj1=JTLReader("C:\\Karthik\\03_Results\\Scenario_ET001_Temp.jtl") 
#obj1.converTimeStamp()
obj1.ms_to_sec() 
obj1.getDuration()



Wall time: 300 ms


'08:04:59'

In [130]:
y=obj1.getTotalPassFailFreqList()
y

,success
True,96863


In [136]:
x=obj1.temp()
x

min        mean  max  count
label                     timeStamp                                       
BP02_eIACSTransactionTime 2020-05-13 19:00:00   13   23.881537  187   2524
                          2020-05-13 19:30:00   12   23.286541  383   2764
                          2020-05-13 20:00:00   14   22.762577  168   2763
                          2020-05-13 20:30:00   13   23.205425  284   2765
                          2020-05-13 21:00:00   13   23.453329  228   2764
                          2020-05-13 21:30:00   14   23.231270  199   2763
                          2020-05-13 22:00:00   14   23.321157  167   2765
                          2020-05-13 22:30:00   14   23.217155  168   2763
                          2020-05-13 23:00:00   13   23.444846  163   2765
                          2020-05-13 23:30:00   13   23.752894  174   2764
                          2020-05-14 00:00:00   14   23.732078  155   2762
                          2020-05-14 00:30:00   13   24.092224  213   2765
                          2020-05-14 01:00:00   13   24.094824  166   2763
                          2020-05-14 01:30:00   13   23.713097  169   2764
                          2020-05-14 02:00:00   14   23.549005  554   2765
                          2020-05-14 02:30:00   13   23.918929  164   2763
                          2020-05-14 03:00:00   14   23.090909  145    495
BP01_BikeCounterStatus    2020-05-13 19:00:00   15   33.129314  487   2521
                          2020-05-13 19:30:00   16   35.066594  491   2763
                          2020-05-13 20:00:00   16   36.764493  480   2760
                          2020-05-13 20:30:00   16   35.828385  438   2762
                          2020-05-13 21:00:00   15   36.765665  471   2761
                          2020-05-13 21:30:00   16   38.622238  569   2761
                          2020-05-13 22:00:00   16   40.898986  575   2762
                          2020-05-13 22:30:00   16   39.447101  555   2760
                          2020-05-13 23:00:00   16   40.910145  544   2760
                          2020-05-13 23:30:00   16   40.791304  547   2760
                          2020-05-14 00:00:00   16   43.965217  590   2760
                          2020-05-14 00:30:00   16   44.281624  598   2759
                          2020-05-14 01:00:00   16   45.312545  554   2758
                          2020-05-14 01:30:00   16   46.915942  691   2760
                          2020-05-14 02:00:00   16   46.812545  648   2758
                          2020-05-14 02:30:00   16   48.421892  632   2759
                          2020-05-14 03:00:00   16   50.080508  627    472
BP03_iBorders             2020-05-13 19:00:00  212  221.216630  337    457
                          2020-05-13 19:30:00  212  220.981928  336    498
                          2020-05-13 20:00:00  212  220.969758  322    496
                          2020-05-13 20:30:00  213  221.337349  348    498
                          2020-05-13 21:00:00  213  222.558468  339    496
                          2020-05-13 21:30:00  213  222.324597  344    496
                          2020-05-13 22:00:00  213  222.050201  340    498
                          2020-05-13 22:30:00  213  223.239919  336    496
                          2020-05-13 23:00:00  212  221.129032  354    496
                          2020-05-13 23:30:00  212  223.514056  831    498
                          2020-05-14 00:00:00  212  222.148893  359    497
                          2020-05-14 00:30:00  213  222.297787  459    497
                          2020-05-14 01:00:00  212  221.509054  333    497
                          2020-05-14 01:30:00  213  220.340040  343    497
                          2020-05-14 02:00:00  212  220.692153  368    497
                          2020-05-14 02:30:00  212  221.839034  343    497
                          2020-05-14 03:00:00  214  221.430380  307     79

In [74]:
#obj1.df_Jtl

In [46]:
obj1.df_jtl_HTTPReqeust

,timeStamp,elapsed,label,responseCode,responseMessage,dataType,success,bytes,allThreads,URL
0,2020-05-13 19:00:22,48,BP02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...
1,2020-05-13 19:00:22,48,BP01_BikeCounterStatus-0,200,OK,text,True,241,3,http://192.168.161.119/bikes/sib/v1/counterstatus
3,2020-05-13 19:00:22,239,BP03_iBorders-0,200,OK,text,True,232,3,http://192.168.161.119/iborders/sib/v1/transac...
5,2020-05-13 19:00:35,24,BP02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...
7,2020-05-13 19:00:35,191,BP01_BikeCounterStatus-0,200,OK,text,True,241,3,http://192.168.161.119/bikes/sib/v1/counterstatus
...,...,...,...,...,...,...,...,...,...,...
193746,2020-05-14 03:05:21,25,BP02_eIACSTransactionTime-0,200,OK,text,True,239,50,http://192.168.161.119/eiacs/sib/v1/transactio...
193748,2020-05-14 03:05:21,18,BP02_eIACSTransactionTime-0,200,OK,text,True,239,50,http://192.168.161.119/eiacs/sib/v1/transactio...
193751,2020-05-14 03:05:21,19,BP01_BikeCounterStatus-0,200,OK,text,True,241,50,http://192.168.161.119/bikes/sib/v1/counterstatus
193752,2020-05-14 03:05:21,18,BP02_eIACSTransactionTime-0,200,OK,text,True,239,50,http://192.168.161.119/eiacs/sib/v1/transactio...


In [44]:
obj1.df_jtl_TC

,timeStamp,elapsed,label,responseCode,responseMessage,dataType,success,bytes,allThreads,URL
2,2020-05-13 19:00:22,48,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
4,2020-05-13 19:00:22,48,BP01_BikeCounterStatus,200,"Number of samples in transaction : 2, number o...",NaN,True,241,3,NaN
6,2020-05-13 19:00:35,24,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
10,2020-05-13 19:00:42,20,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,7,NaN
13,2020-05-13 19:00:42,21,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,7,NaN
...,...,...,...,...,...,...,...,...,...,...
193747,2020-05-14 03:05:21,25,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,50,NaN
193749,2020-05-14 03:05:21,18,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,50,NaN
193750,2020-05-14 03:05:08,21,BP01_BikeCounterStatus,200,"Number of samples in transaction : 2, number o...",NaN,True,241,50,NaN
193753,2020-05-14 03:05:21,18,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,50,NaN


In [129]:
#obj1.getPeriod()

In [130]:
#obj1.getDuration()

In [131]:
#obj1.getMaxRunningVusers()

In [132]:
#obj1.getTotalThroughput()

In [131]:
analysisSummary=obj1.getAnalysisSummary()
analysisSummary

,Data,Values
0,Period,2020-05-13 19:00:22 : 2020-05-14 03:05:21
1,Duration,08:04:59
2,Maximum Running Vusers,50
3,Total bytes,23183119
4,Average bytes/second,796.698
5,TotalHits,96893
6,Average Hits Per Second,3.32977
7,Total Error,NA
8,Total Pass,success True 96863
9,Total Failed,NA


In [132]:
#responseTimeSummary
responseTimeSummary=obj1.responseTimeSummary()
responseTimeSummary

,#Samples,Min,Average,90 Percentile,Max
label,,,,,
BP02_eIACSTransactionTime,44477.0,12.0,23.534209,26.0,554.0
BP01_BikeCounterStatus,44396.0,15.0,41.008987,31.0,691.0
BP03_iBorders,7990.0,212.0,221.758698,224.0,831.0


In [133]:
passandfail=obj1.getPassFailSummary()
passandfail

success,Pass
label,
BP02_eIACSTransactionTime,44477
BP01_BikeCounterStatus,44396
BP03_iBorders,7990


In [134]:
rs=obj1.getFinalResponseSummary()
rs

KeyError: 'Fail'

In [100]:
runningVuser=obj1.getRunnerVuserGraphData()
runningVuser

,Vusers
timeStamp,
2020-05-13 19:00:00,50
2020-05-13 19:30:00,50
2020-05-13 20:00:00,50
2020-05-13 20:30:00,50
2020-05-13 21:00:00,50
2020-05-13 21:30:00,50
2020-05-13 22:00:00,50
2020-05-13 22:30:00,50
2020-05-13 23:00:00,50


In [101]:
hitsPersecond=obj1.getHitsPersecondGraphData() 
hitsPersecond

,hitsCount
timeStamp,
2020-05-13 19:00:00,5502
2020-05-13 19:30:00,6025
2020-05-13 20:00:00,6019
2020-05-13 20:30:00,6025
2020-05-13 21:00:00,6021
2020-05-13 21:30:00,6020
2020-05-13 22:00:00,6025
2020-05-13 22:30:00,6019
2020-05-13 23:00:00,6021


In [102]:
throughput=obj1.getThrouhputPersecondGraphData()
throughput

,Throughput
timeStamp,
2020-05-13 19:00:00,1316821
2020-05-13 19:30:00,1442015
2020-05-13 20:00:00,1440589
2020-05-13 20:30:00,1442013
2020-05-13 21:00:00,1441069
2020-05-13 21:30:00,1440830
2020-05-13 22:00:00,1442013
2020-05-13 22:30:00,1440589
2020-05-13 23:00:00,1441067


In [103]:
#overAllResponseTimeSummary
overallResponseTime=obj1.getOverallResponseTime()
overallResponseTime

,timeStamp,elapsed,label
2,2020-05-13 19:00:22,48,BP02_eIACSTransactionTime
4,2020-05-13 19:00:22,48,BP01_BikeCounterStatus
6,2020-05-13 19:00:35,24,BP02_eIACSTransactionTime
10,2020-05-13 19:00:42,20,BP02_eIACSTransactionTime
13,2020-05-13 19:00:42,21,BP02_eIACSTransactionTime
...,...,...,...
193747,2020-05-14 03:05:21,25,BP02_eIACSTransactionTime
193749,2020-05-14 03:05:21,18,BP02_eIACSTransactionTime
193750,2020-05-14 03:05:08,21,BP01_BikeCounterStatus
193753,2020-05-14 03:05:21,18,BP02_eIACSTransactionTime


In [104]:
writer = ExcelWriter('JTL_Log_Analyser_Output.xlsx',mode='w')
#Write Analysis Summary to Excel
analysisSummary.to_excel(writer,sheet_name='AnalysisSummary')
#write response summary to excel with txn name,min,avg,max,90% & max
#rs.to_excel(writer,sheet_name='ResponseSummary')
#Write Running Vusers
runningVuser.to_excel(writer,sheet_name='RunningVuserSummary')
#Write Hits Summary
hitsPersecond.to_excel(writer,sheet_name='HitsSummary')
#Write Throughput Summary
throughput.to_excel(writer,sheet_name='ThroughputSummary')
#Write ResponseTime
overallResponseTime.to_excel(writer,sheet_name='ResponseTimeGraph')
writer.save()







In [94]:
responseMessage=obj1.getResponseMessage()
responseMessage
#responseMessage_grp=responseMessage.groupby('label',observed=True )
#errorSummary=responseMessage_grp['responseMessage'].value_counts()
#errorSummary.unstack()
#responseMessage=responseMessage.set_option('display.max_colwidth',1000)
#x=responseMessage[['label','responseMessage']]
#x.loc[227077,'responseMessage']

#responseMessage['responseMessage']
#'Non HTTP response message: Address already in use: connect'





,timeStamp,elapsed,label,responseCode,responseMessage,dataType,success,bytes,allThreads,URL


In [ ]:
import random
import matplotlib.pyplot as plt
#from matplotlib import style
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
%matplotlib inline


#print(plt.style.available)
#print(plt.__file__)

#style.use('dark_background')
plt.rcParams['figure.figsize']=[25,18]
fig = plt.figure(figsize=(20,10))
fig.set_facecolor('#003366')

# Add suplot syntax

ax1 = fig.add_subplot(211)

#To get data for x and y axis
x,y=Throughput.index,Throughput['Throughput']
#To rotate 
for label in ax1.xaxis.get_ticklabels():
    label.set_rotation(0)

#To print x axis with date and time
xfmt = mdates.DateFormatter('%H:%M:%S')#%d-%m-%y %H:%M:%S//ax.xaxis_date()
ax1.xaxis.set_major_formatter(xfmt)
ax1.fill_between(x,y,color="red",linewidth=5)#https://matplotlib.org/users/dflt_style_changes.html

# x and y label
ax1.set_xlabel('Time',fontsize=20,color='white')#alpha=0.3
ax1.set_ylabel('Throughput/sec',fontsize=20)
ax1.set_title('Throughput',fontsize=20,color='white')

#To drwa 4 lines with the desier colour : Top,Bottom,Left and Right
ax1.spines['bottom'].set_color('black')
ax1.spines['top'].set_color('black')
ax1.spines['left'].set_color('black')
ax1.spines['right'].set_color('black')

#
ax1.xaxis.label.set_color('white')#"Time"
ax1.yaxis.label.set_color('white')#"HitsPerMin"

ax1.tick_params(axis='x', labelcolor='white',labelsize=20)#"%d-%m-%y %H:%M:%S"
ax1.tick_params(axis='y', labelcolor='white',labelsize=20)#"Hits Count: 10,20,30,40,50,60,70"

ax1.set_facecolor('#003366')#https://matplotlib.org/users/colors.html

##003366

ax1.grid(which='major', linestyle='-', linewidth='0.2', color='white')#17becf
#ax1.grid(which='minor', linestyle=':', linewidth='0.5', color='red')

#ax1.set_yticks([0,1,2,3,4,5,6,7,8,9,10])
#ax1.set_xticks(['22:00:00','22:30:00','23:00:00','23:30:00','00:00:00',
               # '00:30:00','01:00:00','01:30:00','02:00:00','02:30:00',
               # '03:00:00','03:30:00','04:00:00','04:30:00'])
#ax.set_ylim((0, 6))
#ax.set_xlim()

#fig.savefig("E:\MaxUser.pdf",dpi=None,facecolor='#003366', edgecolor='#003366',bbox_inches='tight')
plt.show()